# Spinning Up EMR Cluster

In the last post, we went over briefly some examples of distributed computing and narrowly surveyed the landscape of EMR and Hadoop. In this post, I want to try to actually spin up a cluster. Don't really know what else to say at this point because I'm so uneducated in the area and have read about as much as I could on the product... so let's just dive right into it.

<img src="https://thechive.files.wordpress.com/2015/05/people-who-fail-at-pools-15-gifs-11.gif" width="300">

## AWS EMR Console
Let's open up EMR and try to create a cluster. 4 sections are presented to us:

### General Configuration
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/3_aws_emr_1.png" width="700">

Pretty simple here.
- I'll keep the default name
- I'll turn off logging (this feature enables automatic logging to _**S3**_), note that by turning this off, I'll still have local logs on the EC2s themselves
- I'll keep launch mode to _**cluster**_ by default as well, step execution indicates you want to turn up the cluster, run some automated package and terminate the cluster all in one fell swoop, I'd like to play around with the cluster live

### Software Configuration
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/3_aws_emr_2.png" width="700">

Okay, a bit more complicated here because I'm not 100% familiar with all the services yet haha. It's pretty obvious I'd like to use the Spark option as that's what I set out to do. It looks like EMR can be sliced and diced in quite a few ways. _**Core Hadoop**_ seems to provide us with HDFS and HIVE, and beyond that, I'm not really sure what the other services are. _**HBase and Presto**_ I absolutely don't feel comfortable speaking to, so I'm going to skip these altogether.

The Spark option gives us:
- Spark
- Hadoop (HDFS)
- YARN
- Ganglia (Cluster resource monitoring)
- Zeppelin (A Spark-compatible notebook, similar to Jupyter)

This one seems to have all the stuff we just went over plus some bells and whistles as tools layered on top of Hadoop and Spark. Let's go with this.

Oh yeah, I also have no clue what AWS Glue Metastore is so I'm going to ignore this too (a theme is starting to develop, no?). This blog is after all about trying, _**FAILING**_, and sometimes succeeding at data science.

![](https://media.tenor.com/images/06952aa1867a04dd557b804b5921f849/tenor.gif)

### Hardware Configuration
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/3_aws_emr_3.png" width="700">

These options are relatively simple as well, but we need to get into a bit of math here and review the scope of our task. Our data, again, is around 1.5GB (yes, we've already made the argument that using EMR for this is slightly overkill), so I don't think we need any more than 2 worker nodes. Worker nodes are defined as "Core" nodes within EMR. The default of 1 master + 2 workers sounds good to me.

Now, of what type? The default instance type is a general purpose m3.xlarge (4 CPU, 15 GB RAM, 26 cents / hr). 2 of these guys as workers are probably overkill, not to mention the 3 nodes that will be spun up from this will cost ~80 cents / hr. Not sure if I want to spend a dollar an hour on this just yet.

<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/7_emr_on_demand+pricing_1.png" width="500">
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/8_emr_on_demand+pricing_2.png" width="500">

Let's take a look at the cheapest instance in the general purpose category of EC2s: m4.large. I'm now ignoring m3 altogether because m3 gives you an SSD, which I will not need because I'll theoretically be loading data from S3 straight into the cluster's RAM resources.

The m4.large has 2 CPUs and 8GB RAM per node, and costs _**10 cents / hr**_. I think I can deal with this, although at _**30 cents an hour (1 master + 2 worker)**_, that's still a bit steeper than I'd like. Remember when we [rented out the p2.xlarge box to run our Neural Network](https://strikingmoose.com/2017/08/22/chi-larissa-face-detection-7-cutting-cloud-costs-with-infrastructure-automation-part-ii-ec2-configuration-automation/), we were paying about 20 cents / hr for our spot instance. Actually, that reminds me... I was looking at the on-demand prices for the m4.xlarge. A spot instance actually comes out to around _**2.8 cents / hr**_. I don't really have to be a data scientist to do that math right there:

$$10\ cents\ /\ hr-2.8\ cents\ \ hr=OH\ MY\ GOD\ SIGNIFICANT\ SAVINGS$$

<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/9_emr_spot_pricing.png" width="600">

Seriously though, at 28% of the original price (72% discount), it really makes a difference if we started using EMR often in the future. I'm not really going into production, so if I lose a spot instance, all good, whatever. Let's go with that for now. I really don't know what the overhead of SPARK and YARN will be, but I truly believe the m4.large will be just fine.

One problem though - I don't see an option to request spot instances as my nodes, but I've read multiple blogs of folks who have used spot instances for EMR nodes.

-- 5 minutes later --

Ah, okay, we have to go into advanced options... ugh. Alright well let's just finish the next section and then I guess I'll have to dive into the advanced options.

### Security & Access
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/3_aws_emr_4.png" width="700">

This one is relatively easy as well. I'm going to use my ec2-user ssh key that I already [created in previous projects](https://strikingmoose.com/2017/08/22/chi-larissa-face-detection-6-cutting-cloud-costs-with-infrastructure-automation-part-i-aws-infrastructure-automation/). I won't play around with the IAM roles either because I don't really care about security right now. I just want a cluster up and running.

## AWS EMR Console - Advanced Options
Because I want to optimize on cost, I'm going to go ahead and explore the advanced options. I'll skim what I don't know / need and focus on the spot instance.

### Software & Steps
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/10_emr_aws_emr_advanced_1.png" width="800">

Nothing too crazy here other than us being able to actually choose the packages we want. I've selected everything that was in the Spark package + Hue if I so choose to use HDFS or HIVE.

I will ignore the rest of the options right now, I don't think they're needed.

### Hardware
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/11_emr_aws_emr_advanced_2.png" width="800">

This is exactly what I was looking for. First of all, I can actually choose the VPC and security group here as well. I'm going to choose the ones that [I created with my Terraform script](https://strikingmoose.com/2017/08/22/chi-larissa-face-detection-6-cutting-cloud-costs-with-infrastructure-automation-part-i-aws-infrastructure-automation/) in my previous project. These VPCs and Security Groups are wide open and anyone on the internet can access them. This just gives me ease in troubleshooting as this is all open data I'm working with anyways. Nothing really to be compromised here.

Secondly, I'm able to choose spot instance now and I've set it to 3 cents / hr. Got my 3 m4.large's and I should be off and running. Everything else I've left as default.

### General Cluster Settings
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/12_emr_aws_emr_advanced_3.png" width="800">

I don't see the anything here that I need to tamper with either other than turning off logging to save myself unnecessary usage of s3 space.

### Security
<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/13_emr_aws_emr_advanced_4.png" width="800">

Here, I've assigned my ssh key again, ignored the IAM users, and directed the clusters to sit in [my pre-defined security groups](https://strikingmoose.com/2017/08/22/chi-larissa-face-detection-6-cutting-cloud-costs-with-infrastructure-automation-part-i-aws-infrastructure-automation/) so, again, I don't have to deal with restricting ports and IPs. WIDE OPEN BABY.

## Spinning Up The Cluster
Welp... without further ado... let's spin up the cluster! Actually, this post has been long enough. Let's do it in the next post.